# Import

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import lxml
import sys
import time
import re
import urllib.request

# chromedriver 위치 설정

In [6]:
chromedriver_dir = '/Users/mongkyo/projects/mango-plate/Crawling/chromedriver'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
# driver = webdriver.Chrome(chormedriver_dir, chrome_option=options)

# for문을 사용하여 성수역 검색결과 1페이지 식당 들어갔다 나오기

### 식당 별 for문 돌면서 딕셔너리에 값 가져오기. (리스트 안에 디셔너리들을 넣음)

In [7]:
store_info = []
info = dict()

In [61]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://www.mangoplate.com/')
all_iframes = driver.find_elements_by_tag_name("iframe")
if len(all_iframes) > 0:
    print("Ad Found\n")
    driver.execute_script("""
        var elems = document.getElementsByTagName("iframe"); 
        for(var i = 0, max = elems.length; i < max; i++)
             {
                 elems[i].hidden=true;
             }
                          """)
    print('Total Ads: ' + str(len(all_iframes)))
else:
    print('No frames found')
driver.get('https://www.mangoplate.com/')
elem = driver.find_element_by_name("main-search")
elem.send_keys("성수역")
elem.send_keys(Keys.RETURN)
for page in range(1, 2+1):
    if page == 1:
        pass
    else:
        driver.find_element_by_xpath(f'/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{page}]').click()
        time.sleep(3)
    for li in range(1,10+1):
        for div in range(1,2+1):
            driver.find_element_by_xpath(f'/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{li}]/div[{div}]/figure/a').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            title = soup.find("h1", {"class": "restaurant_name"})
    #         rating = soup.find("strong", {"class": "rate-point"})
            info = dict()
            info['name'] = title.get_text()
    #         info['평점'] = rating.get_text()
            table = soup.find("tbody")
            for thtd in table.find_all("tr"):
                if thtd.th.get_text() != "메뉴":
                    temp = thtd.th.get_text().replace(' ', '')
                    if temp.replace('\n', '') == '주소':
                        info['address'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '전화번호':
                        info['phone'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '음식종류':
                        info['food_type'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '가격대':
                        info['pricee_level'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '주차':
                        info['parking'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '영업시간':
                        info['Business_hour'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '쉬는시간':
                        info['break_time'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '마지막주문':
                        info['last_order'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '휴일':
                        info['holiday'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '사이트':
                        info['website'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '유튜브':
                        info['youtube'] = thtd.td.get_text().replace('\n', '')
                else:
                    info[thtd.th.get_text()] = thtd.td.get_text().replace('\n', '')
            time.sleep(1)
            map_tag = soup.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')['src']
            p = re.compile(r'center=(?P<longitude>[\d\.]+),(?P<latitude>[\d\.]+)')
            k= re.search(p, map_tag)
            info['longitude'] = k.group('longitude')
            info['latitude'] = k.group('latitude')
            store_info.append(info)
            image_test = []
            for n in range(1, 5+1):
                image_test.append(soup.select_one(f'body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type({n}) > figure > figure > img')['src'])
            count = 1
            for url in image_test:
                restaurant_name = info['name']
                img_url = urllib.request.urlretrieve(url, f'{restaurant_name}-{count}.jpg')
                count += 1
            driver.execute_script("window.history.go(-1)")

Ad Found

Total Ads: 2


# shell_plus에서 DB저장

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import lxml
import sys
import time
import re
chromedriver_dir = '/Users/mongkyo/projects/mango-plate/Crawling/chromedriver'
store_info = []
info = dict()

In [ ]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://www.mangoplate.com/')
all_iframes = driver.find_elements_by_tag_name("iframe")
if len(all_iframes) > 0:
    print("Ad Found\n")
    driver.execute_script("""
        var elems = document.getElementsByTagName("iframe"); 
        for(var i = 0, max = elems.length; i < max; i++)
             {
                 elems[i].hidden=true;
             }
                          """)
    print('Total Ads: ' + str(len(all_iframes)))
else:
    print('No frames found')
driver.get('https://www.mangoplate.com/')
elem = driver.find_element_by_name("main-search")
elem.send_keys("성수역")
elem.send_keys(Keys.RETURN)
for page in range(1, 2+1):
    if page == 1:
        pass
    else:
        driver.find_element_by_xpath(f'/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{page}]').click()
        time.sleep(3)
    for li in range(1,10+1):
        for div in range(1,2+1):
            driver.find_element_by_xpath(f'/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{li}]/div[{div}]/figure/a').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            title = soup.find("h1", {"class": "restaurant_name"})
    #         rating = soup.find("strong", {"class": "rate-point"})
            info = dict()
            info['name'] = title.get_text()
    #         info['평점'] = rating.get_text()
            table = soup.find("tbody")
            for thtd in table.find_all("tr"):
                if thtd.th.get_text() != "메뉴":
                    temp = thtd.th.get_text().replace(' ', '')
                    if temp.replace('\n', '') == '주소':
                        info['address'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '전화번호':
                        info['phone_num'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '음식종류':
                        info['food_type'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '가격대':
                        info['price_level'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '주차':
                        info['parking'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '영업시간':
                        info['Business_hour'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '쉬는시간':
                        info['break_time'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '마지막주문':
                        info['last_order'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '휴일':
                        info['holiday'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '사이트':
                        info['website'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '유튜브':
                        info['youtube'] = thtd.td.get_text().replace('\n', '')
            time.sleep(1)
            map_tag = soup.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')['src']
            p = re.compile(r'center=(?P<longitude>[\d\.]+),(?P<latitude>[\d\.]+)')
            k= re.search(p, map_tag)
            info['longitude'] = k.group('longitude')
            info['latitude'] = k.group('latitude')
            store_info.append(info)
            driver.execute_script("window.history.go(-1)")

In [ ]:
for store in store_info:
     Restaurant.objects.get_or_create(**store)

# 사진 정보 가져오기

In [47]:
image1 = soup.select_one('body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type(1) > figure > figure > img')['src']
save_name1 = f"{info['name']}" + ".jpg"
image2 = soup.select_one('body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type(2) > figure > figure > img')['src']
save_name2 = info['name'] + ".jpg"
image3 = soup.select_one('body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type(3) > figure > figure > img')['src']
save_name3 = info['name'] + ".jpg"
image4 = soup.select_one('body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type(4) > figure > figure > img')['src']
save_name4 = info['name'] + ".jpg"
image5 = soup.select_one('body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type(5) > figure > figure > img')['src']
save_name5 = info['name'] + ".jpg"
save_names = []
for n in range(1, 5+1):
    save_names.append(f"{info['name']}{n}" + ".jpg")

In [48]:
save_names
# image1
# urllib.request.urlretrieve(image1, save_names[1])

['힙스터스1.jpg', '힙스터스2.jpg', '힙스터스3.jpg', '힙스터스4.jpg', '힙스터스5.jpg']

In [45]:
for k in range(1,5+1):
    print(f"image{k}") 
    print(f"save_name{k}.jpg")
    urllib.request.urlretrieve()

image1
save_name1
image2
save_name2
image3
save_name3
image4
save_name4
image5
save_name5


In [42]:
urllib.request.urlretrieve(image1, save_name1)
urllib.request.urlretrieve(image2, save_name2) 
urllib.request.urlretrieve(image3, save_name3)
urllib.request.urlretrieve(image4, save_name4)
urllib.request.urlretrieve(image5, save_name5)


('힙스터스.jpg', <http.client.HTTPMessage at 0x1084820b8>)

In [ ]:
for store in store_info:
     Restaurant.objects.get_or_create(**store)

In [46]:
for k in range(1, 5+1):
    urllib.request.urlretrieve(f'image{k}')

ValueError: unknown url type: 'image1'

In [75]:
urllib.request.urlretrieve(image1, save_name1)
urllib.request.urlretrieve(image2, save_name2)
urllib.request.urlretrieve(image3, save_name3)
urllib.request.urlretrieve(image4, save_name4)
urllib.request.urlretrieve(image5, save_name5)


('힙스터스.jpg', <http.client.HTTPMessage at 0x10e6904a8>)

In [68]:
save_image

'name.jpg'

In [69]:
urllib.request.urlretrieve(image1, save_image)

('name.jpg', <http.client.HTTPMessage at 0x10e69c7f0>)

In [38]:
image = soup.select_one(f'body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type({n}) > figure > figure > img')['src']

In [39]:
image



'https://mp-seoul-image-production-s3.mangoplate.com/354157/550285_1534483734942_45688?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80'

In [71]:
info['name']

'힙스터스'

In [53]:
soup.select_one('body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type(1) > figure > figure > img')['src']

'https://mp-seoul-image-production-s3.mangoplate.com/898096_1534490121750680.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80'

In [57]:
image_test = []
for n in range(1, 5+1):
    image_test.append(soup.select_one(f'body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type({n}) > figure > figure > img')['src'])

In [58]:
image_test

['https://mp-seoul-image-production-s3.mangoplate.com/898096_1534490121750680.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
 'https://mp-seoul-image-production-s3.mangoplate.com/898096_1534490123668216.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
 'https://mp-seoul-image-production-s3.mangoplate.com/354157/1116733_1534490007370_49?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
 'https://mp-seoul-image-production-s3.mangoplate.com/354157/1116733_1534490007371_50?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
 'https://mp-seoul-image-production-s3.mangoplate.com/354157/550285_1534483734942_45688?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80']

In [60]:
image_test = []
for n in range(1, 5+1):
    image_test.append(soup.select_one(f'body > main > article > aside.restaurant-photos > div.list-photo_wrap.owl-carousel.owl-theme > div.owl-wrapper-outer > div > div:nth-of-type({n}) > figure > figure > img')['src'])
count = 1
for url in image_test:
    restaurant_name = info['name']
    img_url = urllib.request.urlretrieve(url, f'{restaurant_name}-{count}.jpg')
    count += 1